<a href="https://colab.research.google.com/github/chennurishirisha/deep_learning/blob/main/OptimizeCNNModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 5.0 MB/s 
     |████████████████████████████████| 1.6 MB 40.4 MB/s 


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.9.2


fashion_mnist = keras.datasets.fashion_mnist

In [4]:
fashion_mnist = keras.datasets.fashion_mnist

In [5]:

(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [6]:
train_images = train_images/255.0
test_images = test_images/255.0

In [7]:
train_images[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [8]:
train_images[0].shape

(28, 28)

In [9]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)


In [10]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [11]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [12]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [16]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 24s]
val_accuracy: 0.8669999837875366

Best val_accuracy So Far: 0.8734999895095825
Total elapsed time: 00h 02m 53s


In [17]:
model=tuner_search.get_best_models(num_models=1)[0]

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 96)        960       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        55360     
                                                                 
 flatten (Flatten)           (None, 36864)             0         
                                                                 
 dense (Dense)               (None, 64)                2359360   
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 2,416,330
Trainable params: 2,416,330
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.fit(train_images,train_labels,epochs = 10,validation_split =0.1,initial_epoch =3)

Epoch 4/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2793 - accuracy: 0.8960 - val_loss: 0.3786 - val_accuracy: 0.8787
Epoch 5/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2615 - accuracy: 0.9031 - val_loss: 0.3543 - val_accuracy: 0.8850
Epoch 6/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2349 - accuracy: 0.9138 - val_loss: 0.4018 - val_accuracy: 0.8745
Epoch 7/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2289 - accuracy: 0.9151 - val_loss: 0.4300 - val_accuracy: 0.8818
Epoch 8/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2213 - accuracy: 0.9184 - val_loss: 0.3900 - val_accuracy: 0.8788
Epoch 9/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2070 - accuracy: 0.9238 - val_loss: 0.4403 - val_accuracy: 0.8813
Epoch 10/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1959 - accuracy: 0.9283 - val_loss: 0.4502 - val_accuracy